In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# Dependencies
from bs4 import BeautifulSoup
import requests
import pymongo
import dns
import datetime
from time import sleep
import random
import json

import pandas as pd

from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager

# Project defined dependencies
import config
from pprint import pprint


In [3]:
# NASA Mars News
url_nasanews = 'https://mars.nasa.gov/news/'

# JPL Mars Space Images - Featured Image
url_spaceimage = 'https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/index.html'

# Mars Facts
url_spacefacts = 'https://space-facts.com/mars/'

# Mars Hemispheres
url_usgs_astro ='https://astrogeology.usgs.gov'

usgs_astro_search = '/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'


In [4]:
news_title = ''
news_p = ''
featured_image_url = ''

In [5]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)


[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 88.0.4324
[WDM] - Get LATEST driver version for 88.0.4324
[WDM] - Driver [C:\Users\melis\.wdm\drivers\chromedriver\win32\88.0.4324.96\chromedriver.exe] found in cache




In [6]:
def get_soup(url):

    browser.visit(url)
    html_listings = browser.html
    # Parse HTML with Beautiful Soup
    soup = BeautifulSoup(html_listings, 'html.parser')
    # Retrieve all elements that contain book information
    soup = BeautifulSoup(html_listings)
    return soup

In [7]:
soup_nasanews = get_soup(url_nasanews)

In [8]:
list_item = soup_nasanews.find('div',class_='slide')
news_title = list_item.find('div',class_='content_title').text.strip()
news_p = list_item.find('div',class_='rollover_description_inner').text.strip()

In [9]:
print(news_title + '\n\n')
print(news_p)

NASA InSight's ‘Mole' Ends Its Journey on Mars


The heat probe hasn’t been able to gain the friction it needs to dig, but the mission has been granted an extension to carry on with its other science.


In [10]:
soup_spaceimage = get_soup(url_spaceimage)

In [11]:
header = soup_spaceimage.find('div',class_='floating_text_area')

In [12]:
featured_image_url = url_spaceimage.replace('index.html','') + header.a['href']

In [13]:
print(news_title, '\n')
print(news_p, '\n')
print(featured_image_url, '\n')


NASA InSight's ‘Mole' Ends Its Journey on Mars 

The heat probe hasn’t been able to gain the friction it needs to dig, but the mission has been granted an extension to carry on with its other science. 

https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/image/featured/mars2.jpg 



In [14]:
# Visit the Mars Facts webpage here and use Pandas to scrape the table containing facts about the planet including Diameter, Mass, etc.
tables = pd.read_html(url_spacefacts)
tables

df = tables[0]
df.columns = ['Fact Label', 'Fact']
df
html_table = df.to_html()

In [15]:
# Use Pandas to convert the data to a HTML table string.
html_table

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Fact Label</th>\n      <th>Fact</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>0</th>\n      <td>Equatorial Diameter:</td>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <th>1</th>\n      <td>Polar Diameter:</td>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <th>2</th>\n      <td>Mass:</td>\n      <td>6.39 × 10^23 kg (0.11 Earths)</td>\n    </tr>\n    <tr>\n      <th>3</th>\n      <td>Moons:</td>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <th>4</th>\n      <td>Orbit Distance:</td>\n      <td>227,943,824 km (1.38 AU)</td>\n    </tr>\n    <tr>\n      <th>5</th>\n      <td>Orbit Period:</td>\n      <td>687 days (1.9 years)</td>\n    </tr>\n    <tr>\n      <th>6</th>\n      <td>Surface Temperature:</td>\n      <td>-87 to -5 °C</td>\n    </tr>\n    <tr>\n      <th>7</th>\n      <td>First Record:</td>\n      <td>2nd millennium BC

In [16]:
soup_marshem = get_soup(url_usgs_astro + usgs_astro_search)


In [17]:
sleep(5)
items = soup_marshem.find_all('div',class_='item')
item_detail_urls = []

In [18]:
for item in items:
    item_detail_urls.append(url_usgs_astro + item.a['href'])

In [19]:
hemisphere_image_urls = []

for url in item_detail_urls:
    title = ''
    img_url = ''        
    
    soup_hemdetails = get_soup(url)
    sleep(5)
    title = soup_hemdetails.find('h2',class_='title').text

    downloads = soup_hemdetails.find('div',class_='downloads')    
    sleep(5)
    for a in downloads.find_all('a'):
        if a.text.lower() == 'original':
            img_url = a['href']

    hemisphere_img_dict = {
        'title': title,
        'img_url': img_url
    }

    hemisphere_image_urls.append(hemisphere_img_dict)

In [21]:
hemisphere_image_urls 

[{'title': 'Cerberus Hemisphere Enhanced',
  'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif'}]

In [ ]:
hemisphere_image_urls